In [17]:
import torch
from ConditionalImitationLearningNetwork import ImitationLearningNetwork
#from CarlaDatasetLoader import CarlaDataset
from utils import get_batch_mask
from torch.utils.data import DataLoader

In [18]:
ds = CarlaDataset("data\data.csv", 1, True)


In [19]:
dl = DataLoader(dataset =CarlaDataset, num_workers=0, batch_size=3)


In [20]:
for index, (img, speed, target_vec, max_vec) in enumerate(dl):
    print(index)

TypeError: object of type 'type' has no len()

In [27]:
from imgaug.augmenters.arithmetic import AdditiveGaussianNoise
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import glob
from imgaug import augmenters as iaa
from torchvision.transforms.transforms import ColorJitter
import h5py
import numpy as np
from torchvision.io import read_image
import pandas as pd
import torch

class CarlaDataset(Dataset):
    def __init__(self, target_dir, is_train=True, sequence_length=200):
        #super(CarlaDataset, self).__init__()
        self.sequence_length = sequence_length
        csv = pd.read_csv(target_dir)
        print(csv.head(5))
        self.speeds = csv["Speed"]
        self.targets = csv[["Steer", "Gas", "Brake"]]
        self.imgs = csv["Target path"]
        self.commandd = csv["High level command"]
        self.is_train = is_train
        self.init_transforms()
    
    def init_transforms(self):
        if self.is_train:
            self.transforms = transforms.Compose([
                transforms.RandomOrder([
                    iaa.GaussianBlur((0.0, 1.5)),
                    iaa.AdditiveGaussianNoise(scale=(0.0, 0.05)),
                    iaa.Dropout( (0.0, 1.0), per_channel=0.05) ,
                    iaa.CoarseDropout(0.005, size_percent=0.1)]),
                transforms.ColorJitter(brightness= 0.35 ),
                transforms.ToTensor()
            ])
        else:
            self.transforms = transforms.Compose([
                transforms.ToTensor()
                ])


    def __getitem__(self, index):

        img = read_image(self.imgs[index])
        #img = self.transforms(img)

        target = self.targets.iloc[index, :]

    
        command = self.commands[index]-2

        target_vec = torch.zeros((4,3) )
        target_vec[command, :] = target[:]

        speed = torch.Tensor( self.speeds[index]/90  )
        mask_vec = torch.zeros((4,3))
        mask_vec[command, :] = torch.ones((1,3))

        return img, speed, target_vec, mask_vec#.reshape(-1),

    
    def __len__(self):
        return len(self.imgs)


In [22]:
x = CarlaDataset("data\data.csv")

      Steer  Gas  Brake  Hand Brake  Reverse Gear  Steer Noise  Gas Noise  \
0 -0.039811  0.5    0.0         0.0           0.0    -0.039811        0.5   
1 -0.035458  0.5    0.0         0.0           0.0    -0.035458        0.5   
2 -0.031573  0.5    0.0         0.0           0.0    -0.031573        0.5   
3 -0.028106  0.5    0.0         0.0           0.0    -0.028106        0.5   
4 -0.025015  0.5    0.0         0.0           0.0    -0.025015        0.5   

   Brake Noise    Position X    Position Y  ...  Platform time  Game Time  \
0          0.0  13088.663086  33021.835938  ...    958258944.0    57151.0   
1          0.0  13118.545898  33022.570312  ...    958259136.0    57218.0   
2          0.0  13148.444336  33023.210938  ...    958259328.0    57285.0   
3          0.0  13178.357422  33023.785156  ...    958259584.0    57352.0   
4          0.0  13208.280273  33024.292969  ...    958259776.0    57419.0   

   Orientation X  Orientation Y  Orientation Z  High level command  Noise 

In [26]:
x[3](return_batch=False)

AssertionError: Expected boolean as argument for 'return_batch', got type <class 'torch.Tensor'>. Call augment() only with named arguments, e.g. augment(images=<array>).

In [ ]:
img, speed, commands_masks = torch.randn((20, 3, 88, 200)), torch.randn(20, 1), get_batch_mask(torch.randint(0, 3, (20,1)))
model = ImitationLearningNetwork()
x = model(img, speed)